##**Question 3:**
##(b)Fine-tune the pre-trained model on the **Question-Answering: SQuAD**

In [ ]:
from huggingface_hub import login
login(token="<My Token ID>")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch

model_name = "meta-llama/Llama-3.2-1B"


Loading Squad Dataset

In [ ]:
!pip install datasets
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load the  dataset
squad_dataset = load_dataset("squad_v2")
print(squad_dataset)

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})


In [ ]:
x = squad_dataset['train']
x

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [ ]:
train_squad, test_squad = train_test_split(squad_dataset['train'], test_size=0.2, random_state=42)


In [ ]:
# Get the first 5 samples from the training dataset
first_5_samples = squad_dataset['train'].select(range(5))

first_5_samples_list = first_5_samples[:]
print(first_5_samples_list)

{'id': ['56be85543aeaaa14008c9063', '56be85543aeaaa14008c9065', '56be85543aeaaa14008c9066', '56bf6b0f3aeaaa14008c9601', '56bf6b0f3aeaaa14008c9602'], 'title': ['Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé'], 'context': ['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born Septemb

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Preprocess the dataset
def preprocess_squad(examples):
    # Tokenize the context and question pair
    tokenized_examples = tokenizer(
        examples['question'], examples['context'], truncation=True, padding='max_length', max_length=512
    )

    # Initialize lists to store the positions
    start_positions = []
    end_positions = []

    for i in range(len(examples['answers'])):
        answer = examples['answers'][i]

        # Check if the answer is empty
        if not answer['answer_start'] or not answer['text']:
            # If no answer exists, append dummy values (e.g., 0)
            start_positions.append(0)
            end_positions.append(0)
            continue

        answer_start = answer['answer_start'][0]  # Assuming there's only one answer per question
        answer_text = answer['text'][0]

        # Find the position of the answer in the tokenized context
        start_position = tokenized_examples.char_to_token(answer_start)
        end_position = tokenized_examples.char_to_token(answer_start + len(answer_text))

        # If we couldn't find the token for the answer, mark the positions as 0
        if start_position is None or end_position is None:
            start_position = end_position = 0

        start_positions.append(start_position)
        end_positions.append(end_position)

    # Add the positions to the tokenized examples
    tokenized_examples['start_positions'] = start_positions
    tokenized_examples['end_positions'] = end_positions

    return tokenized_examples


train_encodings = squad_dataset['train'].shuffle(seed=1).select(range(1000)).map(preprocess_squad, batched=True)
test_encodings = squad_dataset['validation'].shuffle(seed=1).select(range(200)).map(preprocess_squad, batched=True)



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForQuestionAnswering

In [ ]:
model_qa = AutoModelForQuestionAnswering.from_pretrained(model_name)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForQuestionAnswering were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['embed_tokens.weight', 'layers.0.input_layernorm.weight', 'layers.0.mlp.down_proj.weight', 'layers.0.mlp.gate_proj.weight', 'layers.0.mlp.up_proj.weight', 'layers.0.post_attention_layernorm.weight', 'layers.0.self_attn.k_proj.weight', 'layers.0.self_attn.o_proj.weight', 'layers.0.self_attn.q_proj.weight', 'layers.0.self_attn.v_proj.weight', 'layers.1.input_layernorm.weight', 'layers.1.mlp.down_proj.weight', 'layers.1.mlp.gate_proj.weight', 'layers.1.mlp.up_proj.weight', 'layers.1.post_attention_layernorm.weight', 'layers.1.self_attn.k_proj.weight', 'layers.1.self_attn.o_proj.weight', 'layers.1.self_attn.q_proj.weight', 'layers.1.self_attn.v_proj.weight', 'layers.10.input_layernorm.weight', 'layers.10.mlp.down_proj.weight', 'layers.10.mlp.gate_proj.weight', 'layers.10.mlp.up_proj.weight', 'layers.10.post_attention_layernorm.weight', 'layers.10.s

In [ ]:
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer, AutoTokenizer

In [ ]:
model_qa.gradient_checkpointing_enable()
for param in model_qa.base_model.parameters():
    param.requires_grad = False

training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=4,  # Reduced epochs to target 1000 steps
    seed=1,
    fp16=False,
    logging_dir="/kaggle/working/logs",
    logging_steps=100,
    log_level="error",
)


# Define the Trainer
model_qa_finetuned = Trainer(
    model=model_qa,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=test_encodings,
    tokenizer=tokenizer
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
model_qa_finetuned.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss
100,3.878800,2.634984
200,1.636400,0.852152
300,0.448400,0.186643
400,0.096900,0.043052
500,0.026100,0.018259
600,0.014600,0.012186
700,0.010300,0.009604
800,0.008700,0.008250
900,0.007700,0.007564
1000,0.007300,0.007349


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn

TrainOutput(global_step=1000, training_loss=0.6135048796534538, metrics={'train_runtime': 1100.8383, 'train_samples_per_second': 3.634, 'train_steps_per_second': 0.908, 'total_flos': 1.195806978048e+16, 'train_loss': 0.6135048796534538, 'epoch': 4.0})

In [ ]:
model_qa_finetuned.save_model("/kaggle/working/results")

In [ ]:
model_qa = AutoModelForQuestionAnswering.from_pretrained(model_name)
model_qa.config.pad_token_id = tokenizer.pad_token_id

# Load the fine-tuned model
model_qa_finetuned = AutoModelForQuestionAnswering.from_pretrained("/kaggle/working/results")
model_qa_finetuned.config.pad_token_id = tokenizer.pad_token_id

## **Question 4:**

##Calculate the scores for the following metrics on the test splits for the pre-trained (zero-shot) and fine-tuned models.

###**Question-Answering:** squad_v2, F1, METEOR, BLEU, ROUGE, exact-match

In [ ]:
!pip install sacrebleu rouge-score nltk

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d359ec4c4add611f1a8df3e83b80426febe4d7ed1a02d939fc383fda726edf71
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from transformers import pipeline

# Load the QA pipeline for the fine-tuned model
qa_pipeline_finetuned = pipeline("question-answering", model=model_qa_finetuned, tokenizer=tokenizer)

# Pre-trained model (zero-shot evaluation)
qa_pipeline_pretrained = pipeline("question-answering", model=model_qa, tokenizer=tokenizer)

def generate_predictions(pipeline, dataset):
    predictions, references = [], []
    for i in range(300):  # Iterate over the length of the dataset
        question = dataset['question'][i]  # Access the question
        context = dataset['context'][i]    # Access the context
        answer = dataset['answers'][i]['text'][0] if dataset['answers'][i]['text'] else ""  # Ground truth answer
        references.append(answer)

        # Model prediction
        result = pipeline({"question": question, "context": context})
        predictions.append(result['answer'])  # Predicted answer
        if i%15==0:
            print(i)

    return predictions, references

predictions_finetuned, references = generate_predictions(qa_pipeline_finetuned, test_squad)
predictions_pretrained, _ = generate_predictions(qa_pipeline_pretrained, test_squad)

In [ ]:
def exact_match(predictions, references):
    matches = [1 if p == r else 0 for p, r in zip(predictions, references)]
    return sum(matches) / len(matches) * 100

em_finetuned = exact_match(predictions_finetuned, references)
em_pretrained = exact_match(predictions_pretrained, references)

print("Exact Match (Fine-tuned):", em_finetuned)
print("Exact Match (Pre-trained):", em_pretrained)

Exact Match (Fine-tuned): 6.666666666666667
Exact Match (Pre-trained): 2.3333333333333335


In [ ]:
from sklearn.metrics import f1_score

def f1_metric(predictions, references):
    f1_scores = []
    for p, r in zip(predictions, references):
        pred_tokens = set(p.split())
        ref_tokens = set(r.split())
        common_tokens = pred_tokens & ref_tokens

        if len(common_tokens) == 0:
            f1_scores.append(0)
        else:
            precision = len(common_tokens) / len(pred_tokens)
            recall = len(common_tokens) / len(ref_tokens)
            f1_scores.append(2 * (precision * recall) / (precision + recall))
    return sum(f1_scores) / len(f1_scores) * 100

f1_finetuned = f1_metric(predictions_finetuned, references)
f1_pretrained = f1_metric(predictions_pretrained, references)

print("F1 Score (Fine-tuned):", f1_finetuned)
print("F1 Score (Pre-trained):", f1_pretrained)


F1 Score (Fine-tuned): 2.8445865862532527
F1 Score (Pre-trained): 2.7893975706669205


In [ ]:
from sacrebleu import corpus_bleu

def bleu_metric(predictions, references):
    references = [[r] for r in references]  # BLEU expects list of lists
    return corpus_bleu(predictions, references).score

bleu_finetuned = bleu_metric(predictions_finetuned, references)
bleu_pretrained = bleu_metric(predictions_pretrained, references)

print("BLEU (Fine-tuned):", bleu_finetuned)
print("BLEU (Pre-trained):", bleu_pretrained)

BLEU (Fine-tuned): 0.0
BLEU (Pre-trained): 21.3643503198117


In [ ]:
!pip install pycocoevalcap
from pycocoevalcap.meteor.meteor import Meteor

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 15.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 29.7 MB/s eta 0:00:00


In [ ]:
def compute_meteor(pred_texts, label_texts):
    meteor = Meteor()
    # Wrap predictions and references as dictionaries with keys as 'id' (just use index here for simplicity)
    scores = [meteor.compute_score({i: [label]}, {i: [pred]})[0] for i, (label, pred) in enumerate(zip(label_texts, pred_texts))]
    score = np.mean(scores)
    return score

# Compute METEOR scores for both fine-tuned and pre-trained models
meteor_finetuned = compute_meteor(predictions_finetuned, references)
meteor_pretrained = compute_meteor(predictions_pretrained, references)

print("METEOR (Fine-tuned):", meteor_finetuned)
print("METEOR (Pre-trained):", meteor_pretrained)

METEOR (Fine-tuned): 0.01787208928954959
METEOR (Pre-trained): 0.01965397891457173


## **Question 5**

##Calculate the number of parameters in the model after fine-tuning. Does it remain the same as the pre-trained model?

In [ ]:
import torch


def count_parameters(model):
    return sum(p.numel() for p in model.parameters())


pretrained_params = count_parameters(model_qa.base_model)
print(f"Number of parameters in pre-trained model: {pretrained_params}")


finetuned_params = count_parameters(model_qa_finetuned)
print(f"Number of parameters in fine-tuned model: {finetuned_params}")


Number of parameters in pre-trained model: 1235814400
Number of parameters in fine-tuned model: 1235818498


**Answer:**
No, both parameters are not same. The difference in the number of parameters between the pre-trained and fine-tuned models is minimal: 4,098 parameters.



## **Question 6:**

##Push the fine-tuned model to 🤗.

In [ ]:
from huggingface_hub import create_repo, upload_folder


# Save the fine-tuned model and tokenizer
#model_qa_finetuned.save_pretrained("path_to_save_model")
#tokenizer.save_pretrained("path_to_save_model")

# Create a repo on Hugging Face (if not already created)
create_repo("GauthamBot/gautham_fine_tuned", exist_ok=True)

# Upload to Hugging Face
upload_folder(
    folder_path="/kaggle/working/results",
    repo_id="GauthamBot/gautham-fine-tune",
    commit_message="Fine-tuned model on Squad dataset"
)


model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]